# 双対法による制約付きMDPの解法

参考

* [Exploration-Exploitation in Constrained MDPs](https://arxiv.org/abs/2003.02189)
* [CONSTRAINED MARKOV DECISION PROCESSES](https://www-sop.inria.fr/members/Eitan.Altman/TEMP/h.pdf)

[前回](RL_CMDP_explore_exploit.ipynb)は線型計画法によってCMDPを解く方法を見ました．
ラグランジュの未定乗数法を使うと，動的計画法で解くこともできます．
今回は次のアルゴリズムのリグレットの解析をします．
（前回同様，報酬とコストは既知とします）．

![OptDual](figs/OptDual_CMDP.png)


準備：
* Optimisticな制約違反： $\widetilde{g}_k =\left({D}_k q^{\pi_k}\left(\widetilde{p}_k\right)-\alpha\right)$
* 真の制約違反： $g_k =\left(D q^{\pi_k}(p)-\alpha\right)$
* Optimisticな価値：$\tilde{f}_k={c}_k^T q^{\pi_k}\left(\widetilde{p}_k\right)$
* 真の価値：$f_k=c^T q^{\pi_k}$
* 最適価値：$f_{o p t}=V_1^*\left(s_1\right)=c^T q^*$

また，次のFailure Eventを考えます（LPのときとだいたい同じです）．

$$
\begin{aligned}
& F_k^p=\left\{\exists s, a, s^{\prime}, h:\left|p_h\left(s^{\prime} \mid s, a\right)-\bar{p}_h^{k-1}\left(s^{\prime} \mid s, a\right)\right| \geq \beta_{h, k}^p\left(s, a, s^{\prime}\right)\right\} \\
& F_k^N=\left\{\exists s, a, h: n_h^{k-1}(s, a) \leq \frac{1}{2} \sum_{j<k} q_h^{\pi_k}(s, a \mid p)-H \ln \frac{S A H}{\delta^{\prime}}\right\}
\end{aligned}
$$

前回同様，Failure Eventに対して、
$\bar{G}=F^p \bigcup F^N$の補集合としてGood Event$G$を定義します．
このGood Eventが成り立っているとき、[RL_UCB_VI_regret_proof.ipynb](RL_UCB_VI_regret_proof.ipynb)でやったような分散の変形を使うと，

$$
\left|\bar{p}_h^{k-1}\left(s^{\prime} \mid s, a\right)-p_h\left(s^{\prime} \mid s, a\right)\right|=C_1 \sqrt{\frac{p_h\left(s^{\prime} \mid s, a\right) L_{\delta, p}}{n_h^k(s, a) \vee 1}}+\frac{C_2 L_{\delta, p}}{n_h^k(s, a) \vee 1}
$$

となる定数$C_1, C_2 > 0$が存在します。ここで$L_{\delta, p}=\ln \left(\frac{6 S A H K}{\delta}\right)$です。

早速リグレットの解析をします．

## ステップ１：Dual Optimism

Good eventが成立しているとき，任意の$k \in [K]$について，

$$\widetilde{f}_k-f_{o p t} \leq-\lambda_k^T \widetilde{g}_k$$

が成り立ちます．

**証明**
$\pi^*$は実行可能解なので，
$$
\begin{aligned}
f_{\text {opt }}=c^T q^{\pi^*}(p) & \geq c^T q^{\pi^*}(p)+\lambda_k^T\left(D q^{\pi^*}(p)-\alpha\right) \\
& \geq \min _{\pi \in \Delta_A^S, p^{\prime} \in \mathcal{P}_k} {c}^T q^\pi\left(p^{\prime}\right)+\lambda_k^T\left({D} q^\pi\left(p^{\prime}\right)-\alpha\right) \\
& ={c}^T q^{\pi_k}\left(\widetilde{p}_k\right)+\lambda_k^T\left({D} q^{\pi_k}\left(\widetilde{p}_k\right)-\alpha\right) \\
& =\widetilde{f}_k+\lambda_k^T \widetilde{g}_k .
\end{aligned}
$$

## ステップ２：更新がだんだん良くなってることの証明．

任意の$\lambda \in \mathbb{R}_+^I$と$K' \in [K]$について，

$$
\sum_{k=1}^{K^{\prime}}\left(-\widetilde{g}_k^T \lambda_k\right)+\sum_{k=1}^N \widetilde{g}_k^T \lambda \leq \frac{t_\lambda}{2}\left\|\lambda_1-\lambda\right\|_2^2+\frac{1}{2 t_\lambda} \sum_{k=1}^{K^{\prime}}\left\|\widetilde{g}_k\right\|^2
$$

が成立します．

**証明**

証明は[RL_CMDP_zero_duality_gap.ipynb](RL_CMDP_zero_duality_gap.ipynb)に近いです．

任意の$\lambda \in \mathbb{R}_+^I$について，ラグランジュ係数の更新より，

$$
\begin{aligned}
\left\|\lambda_{k+1}-\lambda\right\|_2^2 & =\left\|\left[\lambda_k+\frac{1}{t_\lambda} \widetilde{g}_k\right]_{+}-[\lambda]_{+}\right\|_2^2 \\
& \leq\left\|\lambda_k+\frac{1}{t_\lambda} \widetilde{g}_k-\lambda\right\|_2^2 \\
& =\left\|\lambda_k-\lambda\right\|_2^2+\frac{2}{t_\lambda} \widetilde{g}_k^T\left(\lambda_k-\lambda\right)+\frac{1}{t_\lambda^2}\left\|\widetilde{g}_k\right\|^2 .
\end{aligned}
$$

です．両辺に　$t_\lambda / 2$をかけて，

$$
\begin{aligned}
-\frac{t_\lambda}{2}\left\|\lambda_1-\lambda\right\|_2^2 & \leq \frac{t_\lambda}{2}\left\|\lambda_{K^{\prime}+1}-\lambda\right\|_2^2-\frac{t_\lambda}{2}\left\|\lambda_1-\lambda\right\|_2^2 \\
& \leq \sum_{k=1}^{K^{\prime}} \widetilde{g}_k^T\left(\lambda_k-\lambda\right)+\frac{1}{2 t_\lambda} \sum_{k=1}^{K^{\prime}}\left\|\widetilde{g}_k\right\|^2 .
\end{aligned}
$$
がせりいつします．変形して，

$$
\sum_{k=1}^N\left(-\widetilde{g}_k^T \lambda_k\right)+\sum_{k=1}^N \widetilde{g}_k^T \lambda \leq \frac{t_\lambda}{2}\left\|\lambda_1-\lambda\right\|_2^2+\frac{1}{2 t_\lambda} \sum_{k=1}^{K^{\prime}}\left\|\widetilde{g}_k\right\|^2
$$

が成り立ちます．

## ステップ３：任意の$\lambda$に対してのバウンドの導出

ステップ１と２を合わせると，

$$
\sum_{k=1}^{K^{\prime}}\left(\widetilde{f}_k-f_{o p t}\right)+\sum_{k=1}^{K^{\prime}} \widetilde{g}_k^T \lambda \leq \sum_{k=1}^{K^{\prime}}\left(-\widetilde{g}_k^T \lambda_k\right)+\sum_{k=1}^{K^{\prime}} \widetilde{g}_k^T \lambda \leq \frac{t_\lambda}{2}\left\|\lambda_1-\lambda\right\|_2^2+\frac{1}{2 t_\lambda} \sum_{k=1}^{K^{\prime}}\left\|\widetilde{g}_k\right\|^2
$$

が成立します．両辺に$\sum_{k=1}^{K^{\prime}} g_k^T \lambda$を足し，$ \sum_{k=1}^{K^{\prime}} f_k$を引いた上で変形すると，

$$
\begin{aligned}
& \sum_{k=1}^{K^{\prime}}\left(f_k-f_{o p t}\right)+\sum_{k=1}^{K^{\prime}} g_k^T \lambda \\
& \leq \frac{t_\lambda}{2}\left\|\lambda_1-\lambda\right\|_2^2+\frac{1}{2 t_\lambda} \sum_{k=1}^{K^{\prime}}\left\|\widetilde{g}_k\right\|^2+\sum_{k=1}^{K^{\prime}}\left(g_k-\widetilde{g}_k\right)^T \lambda+\sum_{k=1}^{K^{\prime}}\left(f_k-\widetilde{f}_k\right) \\
& \leq \frac{t_\lambda}{2}\left\|\lambda_1-\lambda\right\|_2^2+\frac{1}{2 t_\lambda} \sum_{k=1}^{K^{\prime}}\left\|\widetilde{g}_k\right\|^2+\sqrt{\sum_{i=1}^I\left(\sum_{k=1}^{K^{\prime}}\left(g_{k, i}-\widetilde{g}_{k, i}\right)\right)^2}\|\lambda\|_2+\sum_{k=1}^{K^{\prime}}\left(f_k-\widetilde{f}_k\right)
\end{aligned}
$$

が成り立ちます．最後の不等式はコーシー・シュワルツです．

それぞれの項をバウンドしていきます．

ここで，$\widetilde{g}_{k, i}=V^{\pi_k}\left(s_1 ; {d}, \widetilde{p}_k\right)-\alpha_i \in\left[-L_\delta^c H, H\right]$であることに注意しましょう．これはコストの範囲が$\left[-L_\delta^c H, H\right]$であり，$\alpha\in [0, H]$であるようなMDPでの価値関数とみなせます．よって，

$$\frac{1}{2 t_\lambda} \sum_{k=1}^{K^{\prime}}\left\|\widetilde{g}_k\right\|^2 \lesssim \frac{H^2 I K}{2 t_\lambda}$$

です．

よって，[RL_useful_lemma.ipynb](RL_useful_lemma.ipynb)のリグレットを使えば，

$$
\begin{aligned}
& \left|\sum_{k=1}^{K^{\prime}}\left(f_k-\widetilde{f}_k\right)\right|=\left| \sum_{k=1}^{K^{\prime}}\left(V^{\pi_k}\left(s_1 ; c, p\right)-\widehat{V}^{\pi_k}\left(s_1 ; {c}, \bar{p}_k\right) \right)\right| \leq \widetilde{\mathcal{O}}\left(\sqrt{S \mathcal{N} H^4 K}+(\sqrt{\mathcal{N}}+H) H^2 S A\right) \\
& \left|\sum_{k=1}^{K^{\prime}}\left(g_{k, i}-\widetilde{g}_{k, i}\right)\right|=\left| \sum_{k=1}^{K^{\prime}}\left(V^{\pi_k}\left(s_1 ; d_i, p\right)-\widehat{V}^{\pi_k}\left(s_1 ; {d}, \bar{p}_k\right) \right)\right| \leq \widetilde{\mathcal{O}}\left(\sqrt{S \mathcal{N} H^4 K}+(\sqrt{\mathcal{N}}+H) H^2 S A\right).
\end{aligned}
$$

が成立します．簡単な変形から，

$$
\sqrt{\sum_{i=1}^I\left(\sum_{k=1}^{K^{\prime}}\left(g_{k, i}-\widetilde{g}_{k, i}\right)\right)^2} \leq \widetilde{\mathcal{O}}\left(\sqrt{I S \mathcal{N} H^4 K}+(\sqrt{\mathcal{N}}+H) \sqrt{I} H^2 S A\right)
$$

であることもわかります．$t_\lambda=\sqrt{\frac{H^2 I K}{\rho^2}}$と合わせると，

$$
\begin{aligned}
& \sum_{k=1}^{K^{\prime}}\left(f_k-f_{\text {opt }}\right)+\sum_{k=1}^{K^{\prime}} g_k^T \lambda \\
& \quad \lesssim\left(\rho+\frac{\|\lambda\|_2^2}{\rho}\right) \sqrt{H^2 I K}+\left(\sqrt{I S \mathcal{N} H^4 K}+(\sqrt{\mathcal{N}}+H) \sqrt{I} H^2 S A\right)\|\lambda\|_2 \\
& \quad+\left(\sqrt{S \mathcal{N} H^4 K}+(\sqrt{\mathcal{N}}+H) H^2 S A\right)
\end{aligned}
$$

が任意の$\lambda \in \mathbb{R}^I_+$について成り立ちます．

## ステップ４：リグレットの解析

**性能についてのリグレット**

ステップ３の結果に$\lambda = 0$を考えれば，

$$
\sum_{k=1}^{K^{\prime}} V^{\pi_k}\left(s_1 ; c, p\right)-V^*\left(s_1\right)=\sum_{k=1}^{K^{\prime}} f_k-f_{o p t} \lesssim \widetilde{\mathcal{O}}\left(\sqrt{S \mathcal{N} H^4 K}+\rho \sqrt{H^2 I K}+(\sqrt{\mathcal{N}}+H) H^2 S A\right)
$$

が成立します．


**制約についてのリグレット**

$i\in [I]$を固定して，

$$
\bar{\lambda}_i= \begin{cases}\rho e_i & {\left[\sum_{k=1}^{K^{\prime}} g_{i, k}\right]_{+} \neq 0} \\ 0 & \text { otherwise }\end{cases}
$$

とします．ここで，$e_i(i)$は$e_i(i)=1$ and $e_i(j)=0$ for $j \neq i$です．
$\rho$はSlater pointの仮定で定義する値です（論文参照）．

定義より，$\left\|\bar{\lambda}_i\right\|_2^2 \leq \rho^2$です．ここで，$\lambda=\bar{\lambda}_i$とすれば，

$$
\begin{aligned}
& \sum_{k=1}^{K^{\prime}}\left(f_k-f_{\text {opt }}\right)+\rho\left[\sum_{k=1}^{K^{\prime}} g_{i, k}\right]_{+} \\
& \leq \widetilde{\mathcal{O}}\left((1+\rho)\left(\sqrt{I S \mathcal{N} H^4 K}+\sqrt{H^2 I K}+(\sqrt{\mathcal{N}}+H) \sqrt{I} H^2 S A\right)\right):=\epsilon(K) .
\end{aligned}
$$

です．これは任意の$i \in [I]$について成立するので，

$$
\begin{aligned}
& \max _{i \in[I]} \sum_{k=1}^{K^{\prime}}\left(f_k-f_{\text {opt }}\right)+\rho\left[\sum_{k=1}^{K^{\prime}} g_{i, k}\right]_{+}=\sum_{k=1}^{K^{\prime}}\left(f_k-f_{\text {opt }}\right)+\rho \max _{i \in[I]}\left[\sum_{k=1}^{K^{\prime}} g_{i, k}\right]_{+} \\
& =\sum_{k=1}^{K^{\prime}}\left(f_k-f_{\text {opt }}\right)+\rho \max _{i \in[I]}\left|\left[\sum_{k=1}^{K^{\prime}} g_{i, k}\right]_{+}\right| \\
& =\sum_{k=1}^{K^{\prime}}\left(f_k-f_{\text {opt }}\right)+\rho\left\|\left[\sum_{k=1}^{K^{\prime}} g_k\right]_{+}\right\|_{\infty} \leq \epsilon(K) .
\end{aligned}
$$

です．
